In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/dataset-01/09_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor

stacking_reg = StackingRegressor(
    estimators=[
        ('RandomForest', RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)),
        ('lr', LinearRegression()),
        ('LGBM', LGBMRegressor(n_estimators=111, max_depth=11, random_state=42)),
        ('xgboost', xgb.XGBRegressor(n_estimators=51, max_depth=8, random_state=42)),
        ('catboost', Catboost(n_estimators=56, depth=7, learning_rate=1, random_state=42)),
    ],
    final_estimator=LinearRegression(),
    n_jobs=-1,
    cv=3
)

stacking_reg.fit(X_train, y_train)

In [9]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0452020169863492

In [10]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [11]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.052318973500294

In [12]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/kaggle/input/dataset-01/09_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/kaggle/input/dataset-01/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_14.csv', index=False)